# Numerov Calculation Script

In [104]:
import numpy as np
import pandas as pd
import scipy.constants as const

## Data Read-In

We need the scan file with the dipole moment and we need our numerov.dat file

In [105]:
df = pd.read_csv("numerov.dat", skiprows = 2, names = ["Distances","Energies","Wf_shifted_0","Wf_shifted_1","Wf_shifted_2","Wf_shifted_3","Wf_shifted_4","Wf_0","Wf_1","Wf_2","Wf_3","Wf_4"], sep="\s+")

eigenvalues = pd.read_csv("numerov.dat", skiprows=1, nrows=1, sep="\s+",names=["a","b","c","d","e","f"])

scan_dat = pd.read_csv("scan_data.dat",sep="\t",names=["Distances", "Energy", "Dipole_vectors_xyz"])

dipole_vectors = scan_dat.loc[:, "Dipole_vectors_xyz"]

# This splits the individual dipole vectors
dipole_vectors_split = dipole_vectors.str.split(r'\s+',expand=True)
dipole_vectors_split = dipole_vectors_split.astype(float)




In [147]:


eigenvalues_numpy = eigenvalues.to_numpy() * 10
eigenvalues_slices = eigenvalues_numpy[0,1:]




# Next of we calculate v_n0

eigenvalue_0 = eigenvalues_slices[0]
print(eigenvalue_0)


# Now we iterate and calculate all our v_n0s

v_n0 = []
for i, eig in enumerate(eigenvalues_slices):
    if i == 0:
        continue
    else:
        v_n0.append((eig - eigenvalue_0)/const.h * 4148)

# we have to turn this into a wavenumber ws

c = const.speed_of_light * 100

v_n = []
for freq in v_n0:
    v_n.append(freq / c)


    
# Select all x,y,z values for our dipole moment

mu_x = dipole_vectors_split.loc[:, 0]
mu_y = dipole_vectors_split.loc[:, 1]
mu_z = dipole_vectors_split.loc[:, 2]



# And get the bond distances

distances = df.loc[:, "Distances"]



# Select the Wave_Functions for the y_0 and y_n --> as there are just 4 we can do this manually

wf_0 = df.loc[:, "Wf_shifted_0"]
wf_1 = df.loc[:, "Wf_shifted_1"]
wf_2 = df.loc[:, "Wf_shifted_2"]
wf_3 = df.loc[:, "Wf_shifted_3"]
wf_4 = df.loc[:, "Wf_shifted_4"]


# Make a integrand function

def calculate_integrand(wf_0,mu_comp,wf_n):
    return np.array(wf_0 * mu_comp*wf_1)

integral_mu_x = []
integral_mu_y = []
integral_mu_z = []
for i in range(1,5):

    wf_0 = df.loc[:, "Wf_shifted_0"]
    wf_n = df.loc[:, f"Wf_shifted_{i}"]
    integral_mu_x.append(np.trapz(calculate_integrand(wf_0,mu_x,wf_n),distances))
    integral_mu_y.append(np.trapz(calculate_integrand(wf_0,mu_y,wf_n),distances))
    integral_mu_z.append(np.trapz(calculate_integrand(wf_0,mu_z,wf_n),distances))


# Now we can calculate the transition dipole moment for n=1-4

mu_n_squared = []

for i in range(4):
    mu_n_squared.append(abs(integral_mu_x[i])**2 + abs(integral_mu_y[i])**2 + abs(integral_mu_z[i])**2)

print(mu_n_squared)

f_n0 = []
for i in range(4):
    f_n0.append((4.702e-7)*mu_n_squared[i]*v_n[i])

print(f_n0)




        


14.84791735342
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
